https://note.nkmk.me/en/python-re-match-object-span-group/

In [61]:
lines = [
    "G1 F1800",
    "G1 E-6.5 F2400",
    "G92 E0",
    "M82",
    "G1 X119.988 Y87.177 F1800 ;this is a comment",
    "G1 E6.5 F2400",
    "M203 X10",
    "M20",
    ";TYPE:Skirt/Brim",
    "No command",
]

In [62]:
import re
from typing import List, Match

In [74]:
commands = {
    "G0": {"X": None, "Y": None, "Z": None, "F": None},  # non Extrusion Move
    "G1": {"X": None, "Y": None, "Z": None, "F": None},  # Extrusion Move
    "G4": None,  # Dwell
    "M82": None,  # E Absolute
    "M203": None,  # Max Feedrate
    "M204": None,  # Starting Acceleration
    "M205": None,  # Advanced Settings
    "M83": None,  # E Relative
    "G20": None,  # Inches
    "G21": None,  # Milimeters
    "G90": None,  # Absolute Positioning
    "G91": None,  # Relative Positioning
    "G92": None,  # Set Position
    ";": None,  # Comment
}

In [75]:
def arg_extract(string: str, key_dict: dict):
    """
    Extract arguments from known command dictionarys.
    converts list of states to trajectory segments

    Parameters
    ----------
    string  :   str
        string of Commands
    key_dict : dict
        dictionary with known commands and subcommands

    Returns
    ----------
    arg_dict : dict
        dictionary with all found keys and their arguments

    """
    arg_dict = dict()  # dict to store found arguments for each key
    matches: List[Match] = list()  # list to store matching keywords

    for key in key_dict.keys():  # look for each key in the dictionary
        match = re.search(key, string)  # regex search for key in string
        if match is not None:
            matches.append(match)  # append found matches

    # check for longest match and remove smaller match
    i = 0
    while i < len(matches):
        a: Match = matches[i]
        for match in matches:
            if a.start() == match.start() and a.end() < match.end():
                matches.remove(a)
        i += 1

    # support for multiple commands or additional comments per line
    match_start_list = [match.start() for match in matches]
    next_larger = lambda lst, num: min(
        filter(lambda x: x >= num, lst), default=len(string)
    )  # function to find next largest occurence in list, default to eol
    comment_begin = min(
        [start.start() for start in list(filter(lambda x: x.group() == ";", [match for match in matches]))],
        default=len(string),
    )  # find first comment, default to eol

    for match in matches:  # iterate through all matches, with next match available
        match_end = match.end()  # find arg beginning by using end of match
        key = match.group()  # get key from match
        arg = None

        match_next_start = next_larger(match_start_list, match_end)  # find arg end by using beginning of next arg
        if key != ";":
            arg = string[match_end:match_next_start]  # slice string
            arg = arg.replace(" ", "")  # remove spaces if argument is not a comment
        else:
            arg = string[match_end:]  # special case for comments where everything coming after match is arg

        if key_dict[key] is not None:  # check for nested commands
            arg = arg_extract(arg, key_dict[key])  # call arg_extract through recursion

        # save matches found outside of comments, not applying for comments
        if match.end() <= comment_begin or key == ";":
            arg_dict[key] = arg  # save argument values in dict
    return arg_dict

In [76]:
arg_extract("G1 Z30 Y20; G0", commands)

{'G1': {'Y': '20', 'Z': '30'}, ';': ' G0'}

In [77]:
arg_extract("G1 Z30 F20 Y20 G0 Z2;pimml ;G0 du bauer", commands)

{'G0': {'Z': '2'},
 'G1': {'Y': '20', 'Z': '30', 'F': '20'},
 ';': 'pimml ;G0 du bauer'}

In [73]:
arg_extract("G1 X5 G10 M205", commands)

{'G1': {'X': '5'}, 'M205': '', 'G10': ''}